<a href="https://colab.research.google.com/github/thatguyandy27/Pytorch-Tutorials/blob/main/Pytorch_Transforms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Lambda
import os
from torch import nn
from torch.utils.data import DataLoader

In [2]:
# transform transforms the data (input) X
# ToTensor takes PIL or numpy data and converts it into FloatTensor
# it also scales images into range [0 -> 1]
ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))

)

# target_transform turns the Y values into something else.
# here we know we have 10 labels so we are transforming into a [0] * 10 array and then setting the value to one where idx = y

100%|██████████| 26421880/26421880 [00:01<00:00, 16664530.92it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 300926.60it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5473552.03it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6794297.35it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backend.mps.is_available()
    else "cpu"
  )

print(f"Using {device} device")

Using cuda device


In [6]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
# test the model
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)

print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


In [10]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [11]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
  print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0202, -0.0037, -0.0133,  ...,  0.0096,  0.0078,  0.0022],
        [ 0.0105, -0.0037, -0.0163,  ..., -0.0268, -0.0141, -0.0202]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([0.0041, 0.0343], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0228,  0.0252, -0.0388,  ..., -0.0175, -0.0221,  0.0024],
        [ 0.0021,  0.0274,  0.0212,  ...,  0.0222,  0.0326,  0.0401]],
       device='cuda:0', grad_fn=<SliceBa

In [12]:
# Autograd

x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w ) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [13]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x79c77a519870>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x79c77a51a770>


In [14]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1474, 0.1880, 0.2477],
        [0.1474, 0.1880, 0.2477],
        [0.1474, 0.1880, 0.2477],
        [0.1474, 0.1880, 0.2477],
        [0.1474, 0.1880, 0.2477]])
tensor([0.1474, 0.1880, 0.2477])
